# 第7回課題
2段階の train, valid, test にわける交差検証の自前の実装
（パッケージにしたり関数にしたりするのは自由です．解答はしていません）

https://scikit-learn.org/stable/modules/cross_validation.html　で様々なパターンについて述べられています

# 注意事項
train, valid, test で取ってくるサンプル（id）が一致しないということを必ず確認してください

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [2]:
data = load_wine()

In [3]:
wine_df = pd.DataFrame(min_max_scaler.fit_transform(data["data"]), columns=data["feature_names"])
wine_df = pd.concat([wine_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
print(wine_df.shape)
wine_df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0


# モデル（今回は決定木を利用）

In [4]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0)

# seed の設定

In [5]:
myseed = 2434
random.seed(myseed)
np.random.seed(seed=myseed)

# k-fold の k は変数で扱うこと

In [6]:
k = 5

# train (valid 含む), test の分割

In [7]:

def separate_data(data,label_name,random_seed=2434):
    random.seed(myseed)
    DIVISION_NUM = 5
    test_num = int(len(data)/DIVISION_NUM)
    test_indexes = random.sample([i for i in range(len(data))],test_num)
    train_indexes = list(set([i for i in range(len(data))])-set(test_indexes))
    
    train_data = data.iloc[train_indexes]
    test_data = data.iloc[test_indexes]
    
    return train_data.drop(label_name,axis=1), train_data[label_name], \
           test_data.drop(label_name,axis=1), test_data[label_name]

X_train, y_train, X_test, y_test = separate_data(wine_df,'target',random_seed=2434)

# 交差検証
train, valid を分割しつつ valid をテスト

In [8]:
from sklearn.metrics import accuracy_score # 回答はaccuracy だけにしています．ここも特にしていないです

In [9]:
# k回訓練し，valid で評価

def k_fold_cross_validation(model,x,y,division_num,random_seed=2434):
    random.seed(random_seed)
    unit_record_num = int(len(x)/division_num)
    random_indexes = random.sample([i for i in range(len(x))],len(x))
    separate_indexes = [random_indexes[(i-1) * unit_record_num: i * unit_record_num] for i in range(1,division_num+1)]

    
    #余剰分の末尾への追加
    separate_indexes[len(separate_indexes)-1]\
        .extend(random_indexes[division_num * unit_record_num:len(random_indexes)])
    
    #各分割におけるスコア
    scores = []
    for indexes in separate_indexes:
        
        x_train = x.loc[~x.index.isin(indexes)]
        y_train = y.loc[~y.index.isin(indexes)]
        x_test = x.iloc[indexes]
        y_test = y.iloc[indexes]
        model.fit(x_train,y_train)
        scores.append(accuracy_score(y_test,model.predict(x_test)))
    return scores

In [10]:
# train すべてで訓練し，test を評価

print('validation score: ',np.mean(k_fold_cross_validation(clf,X_train,y_train,k,random_seed=myseed)))
clf.fit(X_train,y_train)
print('test score: ',accuracy_score(y_test,clf.predict(X_test)))

validation score:  0.9792626728110599
test score:  0.9428571428571428


かなりどうでもいい話ですが，indexesとindeciesってどっちの方がメジャーなんでしょう